In [1]:
import * as hardhat from 'hardhat'
import '@nomiclabs/hardhat-ethers'
import { ethers } from 'hardhat'
import {smock} from '@defi-wonderland/smock'

import * as bn from './utils/bn'
import * as evm from './utils/evm'
import * as wallet from './utils/wallet'

In [2]:
let blockNumber = 16000000

await evm.reset({
    jsonRpcUrl: process.env['MAINNET_RPC'],
    blockNumber
})

const provider = ethers.provider

In [3]:
const signer = await ethers.getSigner()
const signerAddress = await signer.getAddress()

In [5]:
await wallet.fund(signer.address, bn.toUnit(1))

In [6]:
const gameAbi = require('./out/ButtPlugWars.sol/ButtPlugWars.json')
const gameFactory = await ethers.getContractFactory(gameAbi.abi, gameAbi.bytecode.object, signer)
const game = await gameFactory.deploy({gasLimit: 10e6})

In [7]:
game.address

0x3D63c50AD04DD5aE394CAB562b7691DD5de7CF6f


In [8]:
await game.tokenURI(0)

{"name" : "Scoreboard", "description" : "ButtPlug Wars scoreboard token", "image_data" : ""}


In [9]:
const chessAbi = require('./out/Game.sol/IChess.json')

const fakeChess = await smock.fake(chessAbi.abi)

In [11]:
fakeChess.board.returns('0x1234')

In [12]:
await fakeChess.board()

BigNumber { _hex: '0x1234', _isBigNumber: true }


# Jeison Testing

In [18]:
const jeisonAbi = require('./out/Jeison.sol/Jeison.json')

In [ ]:
const jsonLibFactory = await ethers.getContractFactory(jeisonAbi.abi, jeisonAbi.bytecode.object, signer)
const jsonLib = await jsonLibFactory.deploy()